# Introduction to numba-dpex

## Sections

- [oneAPI Programming Model Overview](#oneAPI-Programming-Model-Overview)
- [Programming Challenges for Multiple architectures](#Programming-Challenges-for-Multiple-architectures)
- [Introducing oneAPI](#Introducing-oneAPI)
- [Introduction to Intel® Distribution for Python*](#Introduction-to-Intel®-Distribution-for-Python*)
- [Introduction to Numba](#Numba*)
- [Parallelism in Python Using Numba](#Parallelism-in-Python-Using-Numba)
    - _Code:_ [Python code running in serial](#Python-code-running-in-serial)
    - _Code:_ [Numba JIT Compilation for CPU](#Numba-JIT-Compilation-for-CPU)    
- [Introduction to numba-dpex](#Introduction-to-numba-dpex)
    - _Code:_ [numba-dpex Example uisng automatic offload](#Automatic-Offload)  
- _Code:_ [Writing Explicit Kernels in numba-dpex](#Writing-Explicit-Kernels-in-numba-dpex)

## Learning Objectives

* Explain how the __oneAPI__ programming model can solve the challenges of programming in a heterogeneous world
* Introduce Intel® Distribution of Python and numba-dpex
* Utilize __Numba__ and __Numba-dpex__  to write paralle code on CPU and GPU
* Write explicit kernels using numba-dpex @kernel decorator

## oneAPI Programming Model Overview
The __oneAPI__ programming model provides a comprehensive and unified portfolio of developer tools that can
be used across hardware targets, including a range of performance libraries spanning several workload
domains. The libraries include functions custom-coded for each target architecture so the same
function call delivers optimized performance across supported architectures. __SYCL__ is based on
industry standards and open specifications to encourage ecosystem collaboration and innovation.

### oneAPI Distribution
Intel&reg; oneAPI toolkits are available via multiple distribution channels:
* Local product installation: install the oneAPI toolkits from the __Intel® Developer Zone__.
* Install from containers or repositories: install the oneAPI toolkits from one of several supported
containers or repositories.
* Pre-installed in the __Intel® DevCloud__: a free development sandbox for access to the latest Intel® SVMS hardware and select oneAPI toolkits. 

## Programming Challenges for Multiple architectures
Currently in the data centric space there is growth in specialized workloads. Each kind of data centric hardware typically needs to be programmed using different languages and libraries as there is no common programming language or APIs, this requires maintaining separate code bases. Developers have to learn a whole set of different tools as there is inconsistent tool support across platforms. Developing software for each hardware platform requires a separate investment, with little ability to reuse that work to target a different architecture. You will also have to consider the requirement of the diverse set of data-centric hardware.

<img src="Assets/oneapi1.png">


## Introducing oneAPI
__oneAPI__ is a solution to deliver unified programming model to __simplify development__ across diverse architectures. It includes a unified and simplified language and libraries for expressing __parallelism__ and delivers uncompromised native high-level language performance across a range of hardware including __CPUs, GPUs, FPGAs__. oneAPI initiative is based on __industry standards and open specifications__ and is interoperable with existing HPC programming models.

<img src="Assets/oneapi2.png">


## Introduction to Intel® Distribution for Python*

IDP is a complete Python distribution that includes the necessary Python packages to develop high-performing code targeting Intel® XPUs using Python. The distribution includes the following:

* Optimized Python numerical packages NumPy, Scikit-learn, MKL-FFT that use Intel® oneAPI Math Kernel Library (oneMKL) and Intel® oneAPI Data Analytics Library (oneDAL) to offer near-native performance.

* Customized version of the Numba* JIT compiler that allows generating fast code for Intel® XPUs.

* Data Parallel Python that is a set of packages enabling SYCL-based XPU programming.
    1. Data Parallel Control (dpctl): A package for controlling execution on SYCL devices and for SYCL USM data management.
    1. Data Parallel Numeric Python (dpnp): An implementation of the NumPy API using SYCL and oneMKL.
    1. Numba-dpex: A standalone extension to Numba adding SYCL kernel programming to Numba*.

* Faster machine learning with XGBoost*, scikit-learn, and advanced ML usages, including multiple devices, with daal4py.

* Scikit-ipp for image warping, image filtering, and morphological operations. Includes support for transform function multithreading and partial multithreading for filters using OpenMP.

## Numba*

Numba is an open-source, NumPy-aware optimizing compiler for Python developed by Anaconda, Inc in collaboration with an open-source community. It uses the LLVM compiler to generate machine code from Python bytecode.
Numba can compile a large subset of numerically focused Python, including many NumPy functions. Additionally, Numba has support for automatic parallelization of loops, generation of GPU-accelerated code, and creation of ufuncs and C callbacks. For more information about Numba, see the Numba homepage: http://numba.pydata.org

### Parallelising Python Programs Using Numba*

Numba has a built in auto-parallelizer that was contributed by Intel. The auto-parallelizer can be enabled by setting the `parallel=True` option in the `@numba.jit` decorator. The auto-parallelizer analyzes data-parallel code regions in the compiled function and schedules them for parallel execution using either OpenMP or TBB. 

There are two types of operations that Numba can automatically parallelize: Implicitly data-parallel regions such as NumPy array expressions, NumPy ufuncs, NumPy reduction fucntions. Explicitly data-parallel loops that are specified using the `numba.prange` expression. Please refer https://numba.pydata.org/numba-doc/latest/user/parallel.html for further details on Numba's automatic paralleizer.

### Automatic Offload of NumPy Expressions
A key distinction between Numba-dpex and other GPU backends in Numba is the ability to automatically offload specific data-parallel sections of a Numba JIT function.


## Parallelism in Python Using Numba

Python has become a pervasive and useful tool in advancing scientific research and computation. Python has a very rich ecosystem of open-source packages for mathematics, science, and engineering, anchored on the performant numerical computation on arrays and matrices, data analysis and visualization capabilities, and an interactive development environment that enables rapid and collaborative iteration of ideas. Python is used to discover new objects in space, calculate thermodynamics, conduct genomic analysis of cancer, estimate the likelihood of earthquakes, simulate musculoskeletal systems, visualize asteroid chemistries, and much more.

Intel’s accelerated Python packages enable scientists to take advantage of the productivity of Python, while taking advantage of the ever-increasing performance of modern hardware. Intel optimized implementations of NumPy and SciPy leverage the Intel Math Kernel Library to achieve highly efficient multithreading, vectorization, and memory management.
Python as a programming language has enjoyed nearly a decade of use in both industry and academia. This high-productivity language has been one of the most popular abstractions to scientific computing and machine learning, yet the base Python language remains single-threaded. Just how is productivity in these fields being maintained with a single-threaded language?

The Python language’s design, by Guido van Rossum, was meant to trade off type flexibility and predictable, thread-safe behavior against the complexity of having to manage static types and threading primitives. This, in turn, meant having to enforce a global interpreter lock (GIL) to limit execution to a single thread at a time to preserve this design mentality.

Over the last decade, many concurrency implementations have been made for Python―but few in the region of parallelism. Does this mean the language isn’t performant? Let’s explore further.
The base language’s fundamental constructs for loops and other asynchronous or concurrent calls all abide by the single-threaded GIL, so even list comprehensions such as [x*x for x in range(0,10)] will always be single-threaded. The threading library’s existence in the base language is also a bit misleading, since it provides the behavior of a threading implementation but still operates under the GIL. Many of the features of Python’s concurrent futures to almost parallel tasks also operate under the GIL. Why does such an expressive productivity language restrict the language to these rules?

The reason is the level of abstraction the language design adopted. It ships with many tools to wrap C code, from ctypes to cffi. It prefers multiprocessing over multithreading in the base language, as evidenced by the multiprocessing package in the native Python library. These two design ideas are evident in some of the popular packages, like NumPy and SciPy, which use C code under the Python API to dispatch to a mathematical runtime library such as Intel Math Kernel Library (Intel MKL) or OpenBLAS. The community has adopted the paradigm to dispatch to higher-speed C-based libraries, and has become the preferred method to implement parallelism in Python.
In the combination of these accepted methods and language limitations are options to escape them and apply parallelism in Python through unique parallelism frameworks:
* Numba allows for JIT-based compilation of Python code, which can also run LLVM-based Python-compatible code.
* Cython gives Python-like syntax with compiled modules that can target hardware vectorization as it compiles to a C module.
* numexpr allows for symbolic evaluation to utilize compilers and advanced vectorization.

These methods escape Python’s GIL in different ways while preserving the original intent of the language, and all three implement different models of parallelism.
Let’s take the general example of one of the most common language constructs on which we’d want to apply parallelism—the for loop. Looking at the following loop, we can see that it provides a basic service, a vector addition:

### Python code running in serial
The following code demonstrates usage of serial python code. Inspect code; there are no modifications necessary:
1. Inspect the following code cell and click Run (▶)to save the code to file.
2. Next, run (▶) the cell in the __Build and Run__ section following the code to compile and execute the code.

In [ ]:
%%writefile lab/serial_python.py

# Copyright 2021 Intel Corporation
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from numba import njit
import numpy as np
import dpctl
import timeit


def f1(a, b,c,N):
   for i in range(N):    
    c[i] = a[i] + b[i]    
    

N = 500000
a = np.ones(N, dtype=np.float32)
b = np.ones(N, dtype=np.float32)
c = np.zeros(N,dtype=np.float32)

t = timeit.Timer(lambda: f1(a,b,c,N))
print("Time to calculate the sum in Serial",t.timeit(200),"seconds")
print(c)


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_serial_python.sh; if [ -x "$(command -v qsub)" ]; then ./q run_serial_python.sh; else ./run_serial_python.sh; fi

_If the Jupyter cells are not responsive, or if they error out when you compile the code samples, please restart the Jupyter Kernel: 
"Kernel->Restart Kernel and Clear All Outputs" and compile the code samples again_.

### Numba JIT Compilation for CPU
The previous example handles everything sequentially and doesn’t apply any parallelism to the code. Because of the way this code is written, it’s a good candidate for the Numba framework. Numba uses a decorator (with the @ symbol) to flag functions for just-in-time (JIT) compilation, which we’ll try to apply on this function.

The following code demonstrates usage of simple dpex code. Inspect code; there are no modifications necessary:
1. Inspect the following code cell and click Run (▶) to save the code to file.
2. Next, run (▶) the cell in the __Build and Run__ section following the code to compile and execute the code.

In [ ]:
%%writefile lab/simple_njit_cpu.py

from numba import njit,prange
import numpy as np
import dpctl
import timeit

@njit(parallel=True)
def f1(a, b,c,N):
   for i in prange(N):    
    c[i] = a[i] + b[i]     


N = 500000
a = np.ones(N, dtype=np.float32)
b = np.ones(N, dtype=np.float32)
c = np.zeros(N,dtype=np.float32)

t = timeit.Timer(lambda: f1(a,b,c,N))
print("Time to calculate the sum in parallel",t.timeit(200),"seconds")
print(c)


In [ ]:
! chmod 755 q; chmod 755 run_njit_cpu.sh; if [ -x "$(command -v qsub)" ]; then ./q run_njit_cpu.sh; else ./run_njit_cpu.sh; fi


Including the simple decorator increased performance significantly. This works because the original Python code is written in primitives and datatypes that can be easily compiled and vectorized to a CPU.

The lesson here is that achieving parallelism in Python depends on how the original code is written. Cleanliness of datatypes and the use of vectorizable data structures allow Numba to parallelize code with the insertion of a simple decorator. Being careful about the use of Python dictionaries pays dividends, because historically they don’t vectorize well. Generators and comprehensions suffer from the same problem. Refactoring such code to lists, sets, or arrays can facilitate vectorization.

## Introduction to Data Parallel Control (dpctl) 
Dpctl provides a lightweight Python wrapper over a subset of SYCL’s API. The goal of dpctl is not (yet) to provide an abstraction for every SYCL function. Dpctl is intended to provide a common runtime to manage specific SYCL resources, such as devices and USM memory, for SYCL-based Python packages and extension modules.

The main features presently provided by dpctl are:

1. Python wrapper classes for the main SYCL runtime classes mentioned in Section 4.6 of SYCL provisional 2020 spec (https://bit.ly/3asQx07): `platform`, `device`, `context`, `device_selector`, and `queue`.
1. A USM memory manager to create Python objects that use SYCL USM for data allocation.


Dpctl is available as part of the oneAPI Intel Distribution of Python (IDP). Once oneAPI is installed, dpctl is ready to be used by setting up the IDP that is available inside oneAPI. 

## Introduction to numba-dpex

Numba-dpex is a standalone extension to the Numba JIT compiler that adds SYCL programming capabilities to Numba. Numba-dpex is packaged as part of the IDP that comes with oneAPI base toolkit, and you don’t need to install any specific Conda packages.

Numba-dpex provides two ways to express SYCL parallelism:

* __Automatic offload approach via `@numba.jit`__. The automatic approach extends Numba's existing auto-parallelizer to support generating SYCL kernels from data-parallel code regions. Using the automatic offload approach a programmer needs only minimal changes to their existing code and can try to offload an existing `@numba.jit` decorated function to a SYCL device by invoking the function from a `dpctl.device_context`. We will go into the details of the approach later in the tutorial.

* __Explicit kernel written using the `@numba_dpex.kernel` decorator__. The explicit kernel approach is similar to NUmba's other GPU backends, `numba.cuda` and `numba.roc`. The `@numba_dpex.kernel` decorator is provided in the `numba-dpex` package. Several advanced SYCL features such as _indexing_, _synchronization_, _fences_, _atomcis_ are provided by the `@numba_dpex.kernel` decorator. Thus, using the decorator a relatively low-level SYCL kernel can be written directly in Python. The feature is intended for programmers who already have SYCL and GPU programming experience.

### Automatic Offload

The automatic offload feature in numba-dpex is triggered when a `@numba.jit` function is invoked inside a `dpctl.device_context` scope.

## Explicit parallel for-loop

Using Numba's explicit parallel for-loop and the usage of `numba.prange` you can offload the loop to the GPU using 'dpctl.device_context'
The following code demonstrates usage of numba-dpex's explicit offload using dpctl.device_context. Inspect code; there are no modifications necessary:
1. Inspect the following code cell and click Run (▶)to save the code to file.
2. Next, run (▶) the cell in the __Build and Run__ section following the code to compile and execute the code.

In [ ]:
%%writefile lab/simple_context.py
# SPDX-FileCopyrightText: 2020 - 2023 Intel Corporation
#
# SPDX-License-Identifier: Apache-2.0

import dpnp
import numpy.testing as testing

import numba_dpex as ndpx


# Data parallel kernel implementing vector sum
@ndpx.kernel
def kernel_vector_sum(a, b, c):
    i = ndpx.get_global_id(0)
    c[i] = a[i] + b[i]


# Utility function for printing and testing
def driver(a, b, c, global_size):
    kernel_vector_sum[ndpx.Range(global_size)](a, b, c)
    a_np = dpnp.asnumpy(a)  # Copy dpnp array a to NumPy array a_np
    b_np = dpnp.asnumpy(b)  # Copy dpnp array b to NumPy array b_np
    c_np = dpnp.asnumpy(c)  # Copy dpnp array c to NumPy array c_np
    testing.assert_equal(c_np, a_np + b_np)


# Main function
def main():
    N = 10
    global_size = N
    print("Vector size N", N)

    # Create random vectors on the default device
    a = dpnp.random.random(N)
    b = dpnp.random.random(N)
    c = dpnp.ones_like(a)

    print("Using device ...")
    print(a.device)
    driver(a, b, c, global_size)
    print("Done...")


if __name__ == "__main__":
    main()
    

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_simple.sh; if [ -x "$(command -v qsub)" ]; then ./q run_simple.sh; else ./run_simple.sh; fi

## Automatic offload using @njit

The following example demonstrates the auto-offload feature of Numba. Numba automatically identifies all the NumPy function calls within a jit-decorated function that have parallel semantics and compiles them for parallel execution on the CPU or the GPU based on the device context specified using `dpctl.device_context`.

The following code demonstrates usage automatic offload code. Inspect code; there are no modifications necessary:
1. Inspect the following code cell and click Run (▶) to save the code to file.
2. Next, run (▶) the cell in the __Build and Run__ section following the code to compile and execute the code.

In [ ]:
%%writefile lab/auto_njit.py

import dpctl
import numpy as np
import numba
from numba import njit, prange

@numba.njit(parallel=True)
def l2_distance_kernel(a, b):
    sub = a - b
    sq = np.square(sub)
    sum = np.sum(sq)
    d = np.sqrt(sum)
    return d

def main():
    R = 64
    C = 1
    
    X = np.random.random((R,C))
    Y = np.random.random((R,C))
    
    device = dpctl.select_default_device()
    print("Using device ...")
    device.print_device_info()

    with dpctl.device_context(device):
        result = l2_distance_kernel(X, Y)

    print("Result :", result)
    print("Done...")

if __name__ == "__main__":
    main()


In [ ]:
! chmod 755 q; chmod 755 run_njit_auto.sh; if [ -x "$(command -v qsub)" ]; then ./q run_njit_auto.sh; else ./run_njit_auto.sh; fi

#### Controllable the fallback behavior during automatic offload

By default, if a section of code cannot be offloaded to the GPU, it is automatically executed on the CPU and a warning is printed. This behavior is only applicable to JIT functions, auto-offloading of NumPy calls, array expressions and prange loops. To disable this functionality and force code running on GPU, set the environment variable NUMBA_DPEX_FALLBACK_OPTION to false (for example, export NUMBA_DPEX_FALLBACK_OPTION=0). In this case the code is not automatically offloaded to the CPU and errors occur if any.

#### Diagnostic reporting for automatic offload

Export NUMBA_DPEX_OFFLOAD_DIAGNOSTICS=1:

Setting the debug environment variable NUMBA_DPEX_OFFLOAD_DIAGNOSTICS provides emission of the parallel and offload diagnostics information based on produced parallel transforms. The level of detail depends on the integer value between 1 and 4 that is set to the environment variable (higher is more detailed). In the "Auto-offloading" section there is the information on which device (device name) this kernel was offloaded.

## Writing Explicit Kernels in numba-dpex

Writing a SYCL kernel using the `@numba_dpex.kernel` decorator has similar syntax to writing OpenCL kernels. As such, the numba-dpex module provides similar indexing and other functions as OpenCL. The indexing functions supported inside a `numba_dpex.kernel` are:

* numba_dpex.get_local_id : Gets the local ID of the item
* numba_dpex.get_local_size: Gets the local work group size of the device
* numba_dpex.get_group_id : Gets the group ID of the item
* numba_dpex.get_num_groups: Gets the number of gropus in a worksgroup

Refer https://intelpython.github.io/numba-dpex/latest/user_guides/kernel_programming_guide/index.html for more details.


The following code demonstrates usage of simple `@numba_dpex.kernel` code. Inspect code; there are no modifications necessary:
1. Inspect the following code cell and click Run (▶) to save the code to file.
2. Next, run (▶) the cell in the __Build and Run__ section following the code to compile and execute the code.

In [ ]:
%%writefile lab/simple_2d.py
#! /usr/bin/env python

# SPDX-FileCopyrightText: 2020 - 2023 Intel Corporation
#
# SPDX-License-Identifier: Apache-2.0

import dpctl
import dpctl.tensor as dpt
import numpy as np

import numba_dpex as ndpx


@ndpx.kernel
def data_parallel_sum(a, b, c):
    """
    A two-dimensional vector addition example using the ``kernel`` decorator.
    """
    i = ndpx.get_global_id(0)
    j = ndpx.get_global_id(1)
    c[i, j] = a[i, j] + b[i, j]


def driver(a, b, c, global_size):
    data_parallel_sum[global_size](a, b, c)


def main():
    # Array dimensions
    X = 8
    Y = 8
    global_size = ndpx.Range(X, Y)

    a = np.arange(X * Y, dtype=np.float32).reshape(X, Y)
    b = np.arange(X * Y, dtype=np.float32).reshape(X, Y)
    c = np.empty_like(a).reshape(X, Y)

    c = a + b

    device = dpctl.select_default_device()
    a_dpt = dpt.arange(X * Y, dtype=dpt.float32, device=device)
    a_dpt = dpt.reshape(a_dpt, (X, Y))
    b_dpt = dpt.arange(X * Y, dtype=dpt.float32, device=device)
    b_dpt = dpt.reshape(b_dpt, (X, Y))
    c_dpt = dpt.empty_like(a_dpt)
    c_dpt = dpt.reshape(c_dpt, (X, Y))

    print("Using device ...")
    device.print_device_info()

    print("Running kernel ...")
    driver(a_dpt, b_dpt, c_dpt, global_size)
    c_out = dpt.asnumpy(c_dpt)
    assert np.allclose(c, c_out)

    print("Done...")


if __name__ == "__main__":
    main()

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_simple_2d.sh; if [ -x "$(command -v qsub)" ]; then ./q run_simple_2d.sh; else ./run_simple_2d.sh; fi

## Lab Exercise: Matrix multiplication using @kernel.decorator
* In this lab we provide with the source code that computes matrix multiplication using the numba-dpex kernel decorator.
* Follow the __Step1 and Step 2 in the below code__.

1. Select the code cell below, __follow the STEPS 1 and 2__ in the code comments, click run ▶ to save the code to file.
2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [ ]:
%%writefile lab/matrix_mul.py
#! /usr/bin/env python

# SPDX-FileCopyrightText: 2020 - 2023 Intel Corporation
#
# SPDX-License-Identifier: Apache-2.0

import dpctl
import dpctl.tensor as dpt
import numpy as np

import numba_dpex as ndpx

#***Step1: Uncomment the following lines to enable the dpex.kernel decorator***
@ndpx.kernel
def gemm(a, b, c):
    """
    A basic DGEMM implemented as a ``kernel`` function.
    """
    i = ndpx.get_global_id(0)
    j = ndpx.get_global_id(1)
    if i >= c.shape[0] or j >= c.shape[1]:
        return
    c[i, j] = 0
    for k in range(c.shape[0]):
        c[i, j] += a[i, k] * b[k, j]


# Array dimensions
X = 1024
Y = 16
global_size = X, X

griddim = ndpx.Range(X, X)
blockdim = ndpx.Range(Y, Y)


def driver(a, b, c):
    # Invoke the kernel
    gemm[ndpx.NdRange(griddim, blockdim)](a, b, c)


def main():
    a = np.arange(X * X, dtype=np.float32).reshape(X, X)
    b = np.array(np.random.random(X * X), dtype=np.float32).reshape(X, X)
    
    #***Step2: Uncomment the following lines to set the device context and target a GPU***
    device = dpctl.select_default_device()
    a_dpt = dpt.arange(X * X, dtype=dpt.float32, device=device)
    a_dpt = dpt.reshape(a_dpt, (X, X))
    b_dpt = dpt.asarray(b, dtype=dpt.float32, device=device)
    b_dpt = dpt.reshape(b_dpt, (X, X))
    c_dpt = dpt.ones_like(a_dpt)
    c_dpt = dpt.reshape(c_dpt, (X, X))

    print("Using device ...")
    device.print_device_info()

    driver(a_dpt, b_dpt, c_dpt)
    c_out = dpt.asnumpy(c_dpt)

    # Host compute using standard NumPy
    Amat = np.matrix(a)
    Bmat = np.matrix(b)
    Cans = Amat * Bmat

    # Check result
    assert np.allclose(c_out, Cans)

    print("Done...")


if __name__ == "__main__":
    main()

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_matrix_mul.sh; if [ -x "$(command -v qsub)" ]; then ./q run_matrix_mul.sh; else ./run_matrix_mul.sh; fi

# Summary
In this module you will have learned the following:
* How the __oneAPI__ programming model can solve the challenges of programming in a heterogeneous world
* Intel Distribution of Python and its advantages
* How to use __Numba__ and __numba-dpex__  to write paralle code on CPU and GPU
* Write explicit kernels using numba-dpex @kernel decorator